<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=aefb41b56f8017c49947f4dddf4bcfd89cd654df7b820b42d14e82d392c63e5d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-06 10:32:30
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: -48.47 K (-0.34%)
Current PnL: -25.32 L (-16.15%)
CY Booked + Current PnL: -11.03 L (-7.03%)
-------------------
Total profit:  1.89 L
Total loss:  -27.20 L
-------------------
Total Booked + Current PnL: 15.95 L (12.29%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.94%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 90.28 L (62.8%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.37,17.11,2.20,19.69,4940.0,32808.0,224550.0,5.69,72.0,X-MC,3.64,78.0,6.64,1.56,25.07,XY25,NTT,AC
51,NESTLEIND,1377.00,-0.94,14.81,5.74,21.40,17492.0,39312.0,304738.0,6.58,63.0,X-LC,4.99,12.0,2.25,2.12,22.08,XY25,NTT,FMCG
49,MASFIN,398.61,4.23,2.30,19.31,22.05,19354.0,2250.0,100230.0,-11.92,62.0,H-SC,6.55,164.0,0.12,0.70,45.69,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-1.27,-19.67,24.60,0.09,19916.0,-19819.0,80958.0,87.83,42.0,M-SC,3.28,253.0,-1.00,0.56,6.36,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.11,5.00,13.54,19.23,21392.0,7530.0,157990.0,22.81,82.0,L-LC,5.65,179.0,0.35,1.10,27.34,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,12.79,-23.41,82.29,39.61,64598.0,-24000.0,78500.0,-48.16,70.0,H-SC,32.13,135.0,-0.37,0.55,27.06,AR,ATH,MISC
49,MASFIN,398.61,4.23,2.30,19.31,22.05,19354.0,2250.0,100230.0,-11.92,62.0,H-SC,6.55,164.0,0.12,0.70,45.69,XR,ATH,FINANCE
0,5PAISA,593.00,2.20,-26.92,54.49,12.90,87011.0,-58814.0,159682.0,135.58,72.0,H-SC,10.99,161.0,-0.68,1.11,32.77,OX40N,NTT,FINANCE
55,RAJESHEXPO,518.00,1.94,-65.82,192.74,0.07,90380.0,-90285.0,46892.0,1618.43,39.0,L-SC,3.97,268.0,-1.00,0.33,16.62,OX40N,NTT,JEWELLERY
40,INDUSINDBK,1800.00,1.76,-32.39,96.56,32.89,50402.0,-25008.0,52198.0,-744.47,73.0,L-MC,7.20,259.0,-0.50,0.36,43.75,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1952.00,-3.42,-2.16,28.07,25.30,55192.0,-4341.0,196622.0,-5.38,42.0,X-LC,9.03,17.0,-0.08,1.37,31.71,XY25,BTT,REFINERIES
41,INFY,1972.00,-2.07,9.36,22.76,34.24,77876.0,29275.0,342163.0,-14.93,49.0,X-LC,7.41,2.0,0.38,2.38,18.41,X40,NTT,IT
67,SONACOMS,806.63,-1.98,-18.37,70.95,39.54,58586.0,-18585.0,82574.0,-36.10,41.0,M-SC,3.27,220.0,-0.32,0.57,16.44,AR,ATH,AUTO
20,DABUR,735.00,-1.86,0.00,43.70,43.70,108410.0,10.0,248078.0,-8.33,57.0,X-MC,1.91,72.0,0.00,1.73,15.05,XY24,BTT,FMCG
75,TMPV,600.00,-1.86,-24.86,63.67,22.98,108071.0,-56162.0,169736.0,-21.86,55.0,X-LC,4.99,3.0,-0.52,1.18,6.76,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.07,-1.03,110.08,107.91,160851.0,-1522.0,146122.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.02,8.95,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.10,-16.76,20.16,0.02,29293.0,-29256.0,145303.0,121.54,47.0,M-SC,7.20,234.0,-1.00,1.01,26.39,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.27,-19.67,24.60,0.09,19916.0,-19819.0,80958.0,87.83,42.0,M-SC,3.28,253.0,-1.00,0.56,6.36,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.18,-21.07,43.76,13.47,93143.0,-56817.0,212850.0,-67.53,53.0,H-SC,5.37,158.0,-0.61,1.48,8.23,XY24,NTT,PAINTS
66,SIS,528.00,-0.75,-22.89,57.26,21.26,49024.0,-25416.0,85616.0,2021.76,54.0,H-SC,4.31,166.0,-0.52,0.60,15.68,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.79,3.95,16.15,20.74,34109.0,8030.0,211200.0,38.17,74.0,M-MC,6.84,189.0,0.24,1.47,14.99,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.11,5.00,13.54,19.23,21392.0,7530.0,157990.0,22.81,82.0,L-LC,5.65,179.0,0.35,1.10,27.34,XY25,ATH,MINING
84,WIPRO,420.00,0.38,8.40,58.91,72.27,108211.0,14240.0,183688.0,-6.99,55.0,M-LC,5.80,99.0,0.13,1.28,15.10,XR,NTT,IT
49,MASFIN,398.61,4.23,2.30,19.31,22.05,19354.0,2250.0,100230.0,-11.92,62.0,H-SC,6.55,164.0,0.12,0.70,45.69,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.38,8.40,58.91,72.27,108211.0,14240.0,183688.0,-6.99,55.0,M-LC,5.80,99.0,0.13,1.28,15.10,XR,NTT,IT
49,MASFIN,398.61,4.23,2.30,19.31,22.05,19354.0,2250.0,100230.0,-11.92,62.0,H-SC,6.55,164.0,0.12,0.70,45.69,XR,ATH,FINANCE
25,FINCABLES,1641.55,-0.07,-1.03,110.08,107.91,160851.0,-1522.0,146122.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.02,8.95,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.41,-2.38,38.93,35.63,79647.0,-4983.0,204591.0,-16.30,52.0,H-MC,2.98,119.0,-0.06,1.42,13.90,AR,ATH,PHARMA
38,INDIAMART,4810.62,-1.14,-6.58,121.28,106.72,139741.0,-8114.0,115222.0,-53.68,38.0,H-SC,6.86,139.0,-0.06,0.80,15.26,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.26,-14.51,29.59,10.78,60791.0,-34875.0,205443.0,-48.92,14.0,X-LC,4.18,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
23,DMART,5391.45,-0.59,-12.86,48.73,29.60,105985.0,-32106.0,217494.0,-25.71,26.0,X-LC,9.71,19.0,-0.30,1.51,6.49,X40N,ATH,FMCG
22,DIXON,18931.72,-0.74,-17.06,58.27,31.26,111524.0,-39376.0,191392.0,-61.30,30.0,X-MC,17.97,56.0,-0.35,1.33,1.66,X40N,ATH,IT
7,AWL,485.00,-0.40,-24.53,108.28,57.19,266513.0,-79994.0,246133.0,-64.27,31.0,X-MC,8.08,58.0,-0.30,1.71,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.55,-10.51,45.96,30.62,97495.0,-24924.0,212130.0,-36.35,33.0,X-MC,15.85,55.0,-0.26,1.48,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,-0.25,0.79,19.83,20.77,46796.0,1840.0,235984.0,-7.91,38.0,X-MC,0.94,62.0,0.04,1.64,7.76,X40N,NTT,BREWERIES
65,SIEMENS,4671.5,0.04,-17.04,51.29,25.51,79187.0,-31705.0,154390.0,-5.19,44.0,H-LC,1.38,49.0,-0.40,1.07,13.77,AR,ATH,ELECTRICAL
20,DABUR,735.0,-1.86,0.00,43.70,43.70,108410.0,10.0,248078.0,-8.33,57.0,X-MC,1.91,72.0,0.00,1.73,15.05,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-1.15,-6.92,30.14,21.14,63780.0,-15727.0,211612.0,-13.48,38.0,X-MC,2.30,74.0,-0.25,1.47,17.52,XY24,NTT,PAINTS
45,JIOFIN,387.0,-0.10,-3.47,28.96,24.48,74742.0,-9279.0,258086.0,-5.06,52.0,X-LC,2.31,36.0,-0.12,1.80,49.59,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.40,-24.53,108.28,57.19,266513.0,-79994.0,246133.0,-64.27,31.0,X-MC,8.08,58.0,-0.30,1.71,0.00,XY24,NTT,FMCG
43,ITC,452.00,-0.26,-14.51,29.59,10.78,60791.0,-34875.0,205443.0,-48.92,14.0,X-LC,4.18,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
52,PAGEIND,51605.07,-0.55,-10.51,45.96,30.62,97495.0,-24924.0,212130.0,-36.35,33.0,X-MC,15.85,55.0,-0.26,1.48,0.00,X40,ATH,APPARELS
19,COLPAL,3726.84,-0.76,-21.20,79.57,41.51,165140.0,-55825.0,207540.0,-6.08,39.0,X-MC,8.73,61.0,-0.34,1.44,1.07,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.67,-38.67,120.46,35.20,95057.0,-49758.0,78912.0,3.39,42.0,X-SC,17.22,92.0,-0.52,0.55,1.35,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,1.29,-27.30,96.50,42.84,45602.0,-17750.0,47256.0,-52.62,54.0,X-SC,29.25,83.0,-0.39,0.33,6.91,XY24,NTT,MISC
58,RELAXO,1176.00,-0.55,-47.68,191.09,52.31,145137.0,-69208.0,75952.0,-44.34,46.0,X-SC,7.39,91.0,-0.48,0.53,1.67,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.67,-38.67,120.46,35.20,95057.0,-49758.0,78912.0,3.39,42.0,X-SC,17.22,92.0,-0.52,0.55,1.35,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.11,2.18,21.43,24.08,27800.0,2773.0,129723.0,-6.13,58.0,X-SC,6.97,89.0,0.10,0.90,22.23,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.29,-15.93,70.49,43.32,96515.0,-25952.0,136920.0,-27.81,54.0,X-SC,7.76,90.0,-0.27,0.95,5.13,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-1.06,-11.57,36.50,20.71,122083.0,-43763.0,334474.0,-23.95,49.0,X-LC,2.71,1.0,-0.36,2.33,11.76,X40,ATH,IT
41,INFY,1972.00,-2.07,9.36,22.76,34.24,77876.0,29275.0,342163.0,-14.93,49.0,X-LC,7.41,2.0,0.38,2.38,18.41,X40,NTT,IT
75,TMPV,600.00,-1.86,-24.86,63.67,22.98,108071.0,-56162.0,169736.0,-21.86,55.0,X-LC,4.99,3.0,-0.52,1.18,6.76,XY24,NTT,AUTO
81,VBL,671.64,-0.37,-1.65,37.94,35.67,117858.0,-5200.0,310642.0,-13.27,57.0,X-LC,2.41,4.0,-0.04,2.16,11.98,X40N,ATH,FMCG
43,ITC,452.00,-0.26,-14.51,29.59,10.78,60791.0,-34875.0,205443.0,-48.92,14.0,X-LC,4.18,5.0,-0.57,1.43,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.76,-32.39,96.56,32.89,50402.0,-25008.0,52198.0,-744.47,73.0,L-MC,7.20,259.0,-0.50,0.36,43.75,XR,NTT,BANKS
49,MASFIN,398.61,4.23,2.30,19.31,22.05,19354.0,2250.0,100230.0,-11.92,62.0,H-SC,6.55,164.0,0.12,0.70,45.69,XR,ATH,FINANCE
13,BSOFT,831.70,-0.53,-22.96,95.72,50.79,103724.0,-32291.0,108362.0,-1.04,49.0,H-SC,10.73,151.0,-0.31,0.75,26.17,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.10,-16.76,20.16,0.02,29293.0,-29256.0,145303.0,121.54,47.0,M-SC,7.20,234.0,-1.00,1.01,26.39,OX40N,NTT,PAINTS
69,SURYODAY,216.00,-0.36,-19.01,51.91,23.03,75287.0,-34037.0,145034.0,57.89,54.0,H-SC,5.33,167.0,-0.45,1.01,43.90,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,2.20,-26.92,54.49,12.90,87011.0,-58814.0,159682.0,135.58,72.0,H-SC,10.99,161.0,-0.68,1.11,32.77,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,12.79,-23.41,82.29,39.61,64598.0,-24000.0,78500.0,-48.16,70.0,H-SC,32.13,135.0,-0.37,0.55,27.06,AR,ATH,MISC
74,TITAGARH,1548.00,-0.84,-17.53,76.72,45.74,153223.0,-42460.0,199717.0,9.87,58.0,H-SC,2.88,174.0,-0.28,1.39,29.90,XY24,NTT,ENGINEERING
17,COALINDIA,484.83,-0.11,5.00,13.54,19.23,21392.0,7530.0,157990.0,22.81,82.0,L-LC,5.65,179.0,0.35,1.10,27.34,XY25,ATH,MINING
82,VOLTAS,1530.00,1.37,17.11,2.20,19.69,4940.0,32808.0,224550.0,5.69,72.0,X-MC,3.64,78.0,6.64,1.56,25.07,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.00
1,25,44.02
2,50,76.37


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.90
MC    30.90
LC    26.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.60
X40      23.83
X40N     12.72
XY25     10.72
XR        8.97
AR        8.67
OX40N     7.55
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.37
H-SC    22.89
X-LC    22.77
M-SC    11.31
X-SC     7.90
H-MC     4.70
M-MC     1.47
M-LC     1.28
L-LC     1.10
H-LC     1.07
L-SC     0.80
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.90,-8.44,44.02
IT,12.65,-17.60,76.92
FINANCE,11.21,-10.78,57.28
MISC,7.11,-27.20,79.23
ELECTRICAL,5.90,-10.58,51.08
PAINTS,5.69,-12.77,29.52
INSURANCE,4.73,0.30,33.82
PHARMA,3.96,-2.98,35.39
AUTO,2.83,-28.60,73.18


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3149007.0,21
AR,1323924.0,10
XR,1203759.0,12
X40,1095431.0,15
X40N,912870.0,9
OX40N,708092.0,10
XY25,352518.0,7
SR,282848.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3507747.0,24
X-MC,1479697.0,16
M-SC,1432375.0,15
X-LC,952261.0,13
X-SC,783293.0,8
H-MC,403610.0,3
L-SC,176165.0,2
M-LC,108211.0,1
H-LC,79187.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1229415.0      6
           AR         907177.0      5
M-SC       XY24       815597.0      6
H-SC       XR         780720.0      7
X-MC       X40        489409.0      7
           XY24       438703.0      3
X-LC       X40        418668.0      6
X-MC       X40N       381505.0      4
M-SC       OX40N      310125.0      5
H-SC       OX40N      307587.0      4
X-SC       X40N       307522.0      3
           XY24       288417.0      3
H-SC       SR         282848.0      2
X-LC       X40N       223843.0      2
H-MC       AR         209548.0      2
           XY24       194062.0      1
X-SC       X40        187354.0      2
X-LC       XY24       182813.0      2
M-SC       XR         178641.0      2
X-MC       XY25       170080.0      2
M-SC       AR         128012.0      2
X-LC       XY25       126937.0      3
M-LC       XR         108211.0      1
L-SC       OX40N       90380.0      1
           XR          85785.0      1
H-LC       AR          79187.0      1
L-MC       XR          50402.0      1
M-MC       XY25        34109.0      1
L-LC       XY25        21392.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
